In [1]:
%matplotlib inline

import pandas as pd
import nivapy3 as nivapy
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from sqlalchemy import text

plt.style.use('ggplot')

In [2]:
# Connect to Oracle
eng = nivapy.da.connect()

Username:  ···
Password:  ········


Connection successful.


# Biolok: updates for the 2018 report

This notebook describes data processing and RESA2 updates for 2018 Biolok report. Data sources are as follows:

 * All data prior to 2015 are stored in RESA2 as a **dataset** named `BIOLOK rapp2015` (project ID 4030). This dataset comprisies 43 stations, but only a subset of water samples from each site are included. Relevant data can be exported via the RESA2 application, but it is important to **remember to check the box** labelled *Use only water samples from selected projects*, otherwise all samples will be returned. Within the RESA2 database, relevant samples are associated with `SAMPLE_SELECTION_ID` 62 in the `RESA2.SAMPLE_SELECTIONS` table
 
 * From 2015 onwards, the project comprises 42 stations (two have been removed from the original project and one added). Most of the data are stored in both RESA2 and Aquamonitor, but we also receive data from an external laboratory named Faun. Liv Bente has compiled a spreadsheet here
 
         K:\Prosjekter\langtransporterte forurensninger\O-190033-7 BIOLOK_rapp 2014-18\Samlede prøver ulike kilder_til RESA_JES_nov19.xlsx
         
   listing relevant samples since 2015
   
The general workflow is as follows:

 1. Create a new dataset for the 2018 report in the `RESA2.PROJECTS` table
 
 2. Add the 42 stations in the current project to `PROJECTS_STATIONS`
 
 3. Create a new `SAMPLE_SELECTION_ID` in `RESA2.SAMPLE_SELECTION_DEFINITIONS`
 
 4. Update the new sample selection in `RESA2.SAMPLE_SELECTIONS` to include pre-2015 water samples from the 41 "common" stations
 
 5. Check whether all non-Faun samples are already present in RESA, or whether some data need to be transferred from AM
 
 6. Update the new sample selection to include relevant samples from 2015 onwards for the 41 common sites (based on Liv Bente's spreadsheet)
 
 7. Also add relevant samples from all years for Stakksheitjørna, which is the new station added to the project since 2014 (based on Liv Bente's spreadsheet)
 
 8. Add data from Faun to RESA and link to the sample selection (based on Liv Bente's spreadsheet)

## 1. Create dataset

There is no real distinction between "projects" and "datasets" in RESA: both are stored in the `PROJECTS` table. The column named `PROJECT_OR_DATASET` in this table can be used to determine which table the project/dataset is shown in on the *Select project* window of the RESA2 application.

**The new project created below has `PROJECT_ID` 4490**.

In [3]:
## Create a new dataset
#sql = ("INSERT INTO resa2.projects "
#       "  (project_name, contact_person, project_or_dataset, "
#       "   contact_person_initials, project_description) "
#       "VALUES "
#       "  ('BIOLOK rapp2018', 'Liv Bente Skancke', 'Dataset', "
#       "   'LBS', 'Stations included in the Biolok report (data up to 2018; written during 2020)')"
#      )
#eng.execute(sql)

## 2. Add stations

The stations involved in the project since 2015 are listed in the `stations` worksheet of `../project_overview_from_lbs.xlsx`.

In [4]:
# Read stations
stn_df = pd.read_excel('../project_overview_from_lbs.xlsx',
                       sheet_name='stations')
stn_df.head()

,station_id,station_code,station_name,region,biolok_code,nve_vatn_nr,elevation_m,fylke,kommune
0,42,1003-2-4,Saudlandsvatnet,5,V-1,21894,106.0,Vest-Agder,Farsund kommune
1,192,1014-12,Sognevatn,4,IV-9,11078,267.0,Vest-Agder,Vennesla kommune
2,182,1014-25,Drivenesvatn,4,IV-8,11147,176.0,Vest-Agder,Vennesla kommune
3,166,1018-4,Kleivsetvannet,4,IV-10,11592,93.0,Vest-Agder,Søgne kommune
4,118,1034-19,Indre Espelandsvatnet,5,V-2,11095,389.0,Vest-Agder,Hægebostad kommune


In [5]:
## Refromat for adding to database
#stn_df2 = stn_df.copy()
#stn_df2['project_id'] = 4490
#stn_df2['active'] = 'Y'
#stn_df2 = stn_df2[['station_id', 'project_id', 'active', 'biolok_code']]
#stn_df2.rename({'biolok_code':'station_code'}, axis=1, inplace=True)
#
## Write to db
#stn_df2.to_sql('projects_stations', 
#               eng, 
#               schema='RESA2', 
#               if_exists='append', 
#               index=False,
#              )

## 3. Create a new sample selection

In [6]:
## Create sample selection
#sql = ("INSERT INTO resa2.sample_selection_definitions "
#       "  (sample_selection_id, project_id, name) "
#       "VALUES "
#       "  (66, 4490, 'Biolok (start to 2018)')"
#      )
#eng.execute(sql)

## 4. Add pre-2015 water samples

The code below first queries all water samples associated with the 2015 report, which includes two stations that are no longer part of the project. Next, the code queries *all* water samples in the database associated with the 42 stations that *are* in the current project, and then calculates the intersection of these two sets. This produces a list of pre-2015 samples to be transferred.

The pre-2015 samples are associated with `SAMPLE_SELECTION_ID` 62.

In [7]:
# Get WS IDs for pre-2015 stations
sql = ("SELECT water_sample_id FROM resa2.sample_selections "
       "WHERE sample_selection_id = 62")
pre2015_df = pd.read_sql(sql, eng)

# Get all WS in db for stations in 2018 project
bind_pars = ','.join('%d' % i for i in stn_df['station_id'])
sql = ("SELECT water_sample_id FROM resa2.water_samples "
       "WHERE station_id IN (%s)" % bind_pars)
all_df = pd.read_sql(sql, eng)

# Compute intersection
ws_ids = list(set(pre2015_df['water_sample_id']).intersection(set(all_df['water_sample_id'])))

# Build df
ws_df = pd.DataFrame({'water_sample_id':ws_ids})
ws_df['sample_selection_id'] = 66
print(len(ws_df))

ws_df.head()

1566


,water_sample_id,sample_selection_id
0,273550,66
1,273552,66
2,532542,66
3,532544,66
4,532545,66


In [8]:
## Write to db
#ws_df.to_sql('sample_selections', 
#             eng, 
#             schema='RESA2', 
#             if_exists='append', 
#             index=False,
#            )

## 5. Check whether all non-Faun data are already in RESA

Liv Bente's Excel file here

    K:\Prosjekter\langtransporterte forurensninger\O-190033-7 BIOLOK_rapp 2014-18\Samlede prøver ulike kilder_til RESA_JES_nov19.xlsx
    
inludes a worksheet named `Data fra AquaM`, which lists water chemistry data that should be taken from AM. However, it looks as though most of these samples are already available in RESA. I've created a tidied version of Liv Bente's AM samples list in the `lbs_am_samples` worksheet of `../project_overview_from_lbs.xlsx`. The code below loops over these to see whether they can be identified RESA instead.

**Note:** Initially, around 30 water samples could not be identified. Most of these were due to missing links iin the `NIVADATABASE.DATASOURCE_STATION` table, which I have now added. In addition:

 * AM station 26199 (Storbørja) was incorrectly linked to RESA ID 3042 (Svartbørja)
 
 * One sample from Rondvatnet (AM ID 49204; RESA ID 12712) had the date incorrectly assigned in RESA: the date was given as 13.07.2017 in AM, but 13.06.2017 in RESA. I will assume the AM date is correct
 
Having corrected the above issues, all samples in Liv Bente's `Data fra AquaM` worksheet can now be identified in RESA.

In [9]:
# Read list of AM samples
am_ws = pd.read_excel('../project_overview_from_lbs.xlsx',
                      sheet_name='lbs_am_samples')
am_ws.head()

,am_id,resa_code,am_name,resa_name,date,depth1,depth2
0,8614,430-I-1,Atnsjøen,Atnsjøen,2018-06-10 00:00:00,0.5,0.5
1,8614,430-I-1,Atnsjøen,Atnsjøen,2018-08-09 00:00:00,0.5,0.5
2,8614,430-I-1,Atnsjøen,Atnsjøen,2018-09-04 00:00:00,0.5,0.5
3,26075,432-1-26,Måsåbutjønna,Måsabutjørna,2018-06-14 13:00:00,0.5,0.5
4,26075,432-1-26,Måsåbutjønna,Måsabutjørna,2018-08-28 00:00:00,0.5,0.5


In [10]:
# Loop over data
ws_list = []
for idx, row in am_ws.iterrows():
    # Get station ID
    stn_code = row['resa_code']
    stn_id = stn_df.query('station_code == @stn_code')['station_id'].iloc[0]
    
    # Query RESA db
    par_dict = {'stn_id':stn_id.item(),
                'sample_date':  row['date'].date(),
                'depth1':row['depth1'],
                'depth2':row['depth2'],
               }
    
    sql = ("SELECT * FROM resa2.water_samples "
           "WHERE station_id = :stn_id "
           "AND TRUNC(sample_date) = :sample_date "
           "AND depth1 = :depth1 "
           "AND depth2 = :depth2")
    
    res = pd.read_sql(sql, con=eng, params=par_dict)
    
    if len(res) == 1:
        ws_list.append(res['water_sample_id'].iloc[0])
    elif len(res) == 0:
        ws_list.append(np.nan)
    else:
        print(stn_code, row['date'].date(), len(res))
        ws_list.append(-1)

am_ws['resa_ws_id'] = ws_list
am_ws.head()

,am_id,resa_code,am_name,resa_name,date,depth1,depth2,resa_ws_id
0,8614,430-I-1,Atnsjøen,Atnsjøen,2018-06-10 00:00:00,0.5,0.5,661404
1,8614,430-I-1,Atnsjøen,Atnsjøen,2018-08-09 00:00:00,0.5,0.5,661558
2,8614,430-I-1,Atnsjøen,Atnsjøen,2018-09-04 00:00:00,0.5,0.5,661844
3,26075,432-1-26,Måsåbutjønna,Måsabutjørna,2018-06-14 13:00:00,0.5,0.5,661408
4,26075,432-1-26,Måsåbutjønna,Måsabutjørna,2018-08-28 00:00:00,0.5,0.5,661679


In [11]:
am_ws.count()

am_id         85
resa_code     85
am_name       85
resa_name     85
date          85
depth1        85
depth2        85
resa_ws_id    85
dtype: int64

In [12]:
not_found_df = am_ws[pd.isna(am_ws['resa_ws_id'])]
not_found_df

,am_id,resa_code,am_name,resa_name,date,depth1,depth2,resa_ws_id


In [13]:
am_list = list(not_found_df['am_id'].unique().astype(str))
print(am_list)
if len(am_list) > 0:
    am_ids = ','.join(am_list)
    sql = text(f"SELECT * "
               f"FROM nivadatabase.datasource_station "
               f"WHERE station_id IN ({am_ids})")
    pd.read_sql(sql, eng)

[]


## 6. Add post-2014 water samples (non-FAUN)

**Note:** This code has not yet been run fully. Once the issues with Storbørja and Svartbørja have been solved, it can be run again.

All of the non-FAUN samples from 2015 to 2018 should now be available in RESA (i.e. everything from Liv Bente's `Data fra RESA 2015-2018 mm` and `Data fra AquaM` worksheets). All these samples are listed in tidied form in the `am_resa_2015-2018` worksheet of `../project_overview_from_lbs.xlsx`. The code below identifies RESA sample IDs for each sample and checks that all samples can be found. They are then associated with `SAMPLE_SELECTION_ID` 66 in RESA.

In [14]:
# Read list of post-2014 non-FAUN samples
samp_df = pd.read_excel('../project_overview_from_lbs.xlsx',
                        sheet_name='am_resa_2015-2018')
samp_df.head()

,station_id,date,depth1,depth2
0,16131,2015-06-11 08:00:00,0.0,0.0
1,16131,2015-08-04 08:30:00,0.0,0.0
2,16131,2015-08-30 08:30:00,0.0,0.0
3,16131,2015-10-04 09:00:00,0.0,0.0
4,16131,2016-06-08 00:00:00,0.0,10.0


In [15]:
# Loop over data
ws_list = []
for idx, row in samp_df.iterrows():   
    # Query RESA db
    par_dict = {'stn_id':row['station_id'],
                'sample_date':  row['date'].date(),
                'depth1':row['depth1'],
                'depth2':row['depth2'],
               }
    
    sql = ("SELECT * FROM resa2.water_samples "
           "WHERE station_id = :stn_id "
           "AND TRUNC(sample_date) = :sample_date "
           "AND depth1 = :depth1 "
           "AND depth2 = :depth2")
    
    res = pd.read_sql(sql, con=eng, params=par_dict)
    
    if len(res) == 1:
        ws_list.append(res['water_sample_id'].iloc[0])
    elif len(res) == 0:
        ws_list.append(np.nan)
    else:
        print(stn_code, row['date'].date(), len(res))
        ws_list.append(-1)

samp_df['water_sample_id'] = ws_list
samp_df.head()

,station_id,date,depth1,depth2,water_sample_id
0,16131,2015-06-11 08:00:00,0.0,0.0,638920
1,16131,2015-08-04 08:30:00,0.0,0.0,642959
2,16131,2015-08-30 08:30:00,0.0,0.0,640953
3,16131,2015-10-04 09:00:00,0.0,0.0,642399
4,16131,2016-06-08 00:00:00,0.0,10.0,652766


In [16]:
samp_df.count()

station_id         346
date               346
depth1             346
depth2             346
water_sample_id    346
dtype: int64

In [17]:
not_found_df = samp_df[pd.isna(samp_df['water_sample_id'])]
not_found_df

,station_id,date,depth1,depth2,water_sample_id


In [18]:
## Add to sample selection
#samp_df['sample_selection_id'] = 66
#samp_df = samp_df[['water_sample_id', 'sample_selection_id']]
#
## Write to db
#samp_df.to_sql('sample_selections', 
#               eng, 
#               schema='RESA2', 
#               if_exists='append', 
#               index=False,
#              )

## 7. Data from Faun

The worksheet `faun` of `../project_overview_from_lbs.xlsx` list all the FAUN samples in Liv Bente's spreadsheet. I have also created a list of new methods with `LABORATORY=FAUN` in the worksheet `faun_methods`, which have been added to the database.

### 7.1. Link new method to parameters

In [19]:
# Read methods from Excel
meth_df = pd.read_excel('../project_overview_from_lbs.xlsx',
                        sheet_name='faun_methods')

# Get new method IDs from database
sql = text("SELECT wc_method_id, name AS method_name "
           "FROM resa2.wc_method_definitions "
           "WHERE laboratory = 'FAUN'")
meth_id_df = pd.read_sql(sql, eng)

# Join
meth_df = pd.merge(meth_df, meth_id_df, how='left', on='method_name')

# Add corr_fac = 1
meth_df['conversion_factor'] = 1

# Add to db
par_meth_df = meth_df[['wc_parameter_id', 'wc_method_id', 'conversion_factor']]

In [20]:
#par_meth_df.to_sql('wc_parameters_methods', 
#                   eng, 
#                   schema='RESA2', 
#                   if_exists='append', 
#                   index=False,
#                  )

### 7.2. Add water samples

In [21]:
# Read FAUN data
faun_df = pd.read_excel('../project_overview_from_lbs.xlsx',
                        sheet_name='faun')

# Add water samples
ws_df = faun_df[['station_id', 'sample_date', 'depth1', 'depth2']]

In [22]:
#ws_df.to_sql('water_samples', 
#             eng, 
#             schema='RESA2', 
#             if_exists='append', 
#             index=False,
#            )

### 7.3. Add water chemistry

In [23]:
def f(row):
    """ Function to deal with flags.
    """
    if '<' in row['value']:
        val = '<'
    elif '>' in row['value']:
        val = '>'
    else:
        val = np.nan
    return val

In [24]:
# Get ws_ids
stn_list = list(ws_df['station_id'].unique().astype(str))
stn_txt = ','.join(stn_list)

sql = text("SELECT water_sample_id, station_id, sample_date, depth1, depth2 "
           "FROM resa2.water_samples "
           "WHERE station_id in (%s)" % stn_txt)

samp_df = pd.read_sql(sql, eng)

# Link to faun data
df = pd.merge(faun_df, 
              samp_df, 
              how='left', 
              on=['station_id', 'sample_date', 'depth1', 'depth2'],
             )

df.drop(['station_id', 'station_code', 'sample_date', 'depth1', 'depth2'], 
        axis=1, 
        inplace=True)

# Convert to long format
df = df.melt(id_vars='water_sample_id', 
             var_name='method_name').dropna()

# Join method IDs
df = pd.merge(df, 
              meth_df[['method_name', 'wc_method_id']],
              how='left',
              on='method_name',
             )

# Deal with flags
df['value'] = df['value'].astype(str)
df['flag1'] = df.apply(f, axis=1)

# Extract numeric chars
df['value'] = df['value'].str.extract("([-+]?\d*\.\d+|\d+)", expand=True)
df['value'] = df['value'].astype(float)

# Reorder
df = df[['water_sample_id', 'wc_method_id', 'value', 'flag1']]
df.columns = ['sample_id', 'method_id', 'value', 'flag1']
df['approved'] = 'YES'

df.head()

,sample_id,method_id,value,flag1,approved
0,872225,10959,4.9,NaN,YES
1,872226,10959,5.4,NaN,YES
2,872227,10959,5.0,NaN,YES
3,872245,10959,5.2,NaN,YES
4,872246,10959,5.4,NaN,YES


In [25]:
## Add to db
#df.to_sql('water_chemistry_values2', 
#          eng, 
#          schema='RESA2', 
#          if_exists='append', 
#          index=False,
#         )

### 7.4. Add to sample selection

In [26]:
# Build df
ws_df = pd.DataFrame({'water_sample_id':df['sample_id'].unique()})
ws_df['sample_selection_id'] = 66

## Write to db
#ws_df.to_sql('sample_selections', 
#             eng, 
#             schema='RESA2', 
#             if_exists='append', 
#             index=False,
#            )